In [ ]:
import pandas as pd
## Import OIT Target info
df = pd.read_excel("../Data_For_Tableau/Target_Ref2019.xlsx", sheet_name = "OIT_Target")
df

In [ ]:
## Convert from Wide to Long format
oit_long = pd.melt(df, id_vars=['Region'], var_name='Quarter', value_name='OIT_QTarget')
oit_long.head()

In [ ]:
## Create Quarter - Month break up DataFrame for referece
q_m_df = pd.DataFrame([{'Quarter' : 'Q1', 'Month' : 'Jan,Feb,Mar'},
                    {'Quarter' : 'Q2', 'Month' : 'Apr,May,Jun'},
                    {'Quarter' : 'Q3', 'Month' : 'Jul,Aug,Sep'},
                    {'Quarter' : 'Q4', 'Month' : 'Oct,Nov,Dec'}
                   ])
q_m_df

In [ ]:
## Join Quarter month mapping to OIT df to get month CSV
oit_long = pd.merge(oit_long, q_m_df, on='Quarter')
oit_long.head()

In [ ]:
## Split CSV month and expand each quarter to 3 months
oit_long_expanded = pd.DataFrame(oit_long.Month.str.split(',').tolist(), index=[oit_long.Region, oit_long.Quarter, oit_long.OIT_QTarget]).stack()
oit_long_expanded = oit_long_expanded.reset_index()[['Region', 'Quarter', 0, 'OIT_QTarget']]
oit_long_expanded.columns = ['Region', 'Quarter', 'Month', 'OIT_QTarget']
oit_long_expanded.head()

In [ ]:
## Allocationg a 30-30-40 split for each month in a quarter
oit_long_expanded['Monthly_alloc'] = [0.4 if x in ["Mar","Jun", "Sep", "Dec"] else 0.3 for x in oit_long_expanded['Month']]
oit_long_expanded.head()

In [ ]:
## Calculating monthly target by multiplying quaterly target provided originally, with monthly allocation percentage
oit_long_expanded['OIT_MTarget'] = oit_long_expanded.OIT_QTarget * oit_long_expanded.Monthly_alloc
oit_long_expanded.head()

In [ ]:
df1 = pd.read_excel("../Data_For_Tableau/Target_Ref2019.xlsx", sheet_name = "Sales_Target")
df1

In [ ]:
sales_long = pd.melt(df1, id_vars=['Region'], var_name='Quarter', value_name='Sales_QTarget')
sales_long.head()

In [ ]:
sales_long = pd.merge(sales_long, q_m_df, on='Quarter')
sales_long.head()

In [ ]:
sales_long_expanded = pd.DataFrame(sales_long.Month.str.split(',').tolist(), index=[sales_long.Region, sales_long.Quarter, sales_long.Sales_QTarget]).stack()
sales_long_expanded = sales_long_expanded.reset_index()[['Region', 'Quarter', 0, 'Sales_QTarget']]
sales_long_expanded.columns = ['Region', 'Quarter', 'Month', 'Sales_QTarget']
sales_long_expanded.head()

In [ ]:
sales_long_expanded['Monthly_alloc'] = [0.4 if x in ["Mar","Jun", "Sep", "Dec"] else 0.3 for x in sales_long_expanded['Month']]
sales_long_expanded.head()

In [ ]:
sales_long_expanded['Sales_MTarget'] = sales_long_expanded.Sales_QTarget * sales_long_expanded.Monthly_alloc
sales_long_expanded.head()

In [ ]:
## Merge OIT and Sales targets to single df
targets_df = pd.merge(oit_long_expanded, sales_long_expanded, on=['Region','Quarter', 'Month', 'Monthly_alloc'])
targets_df.head()